In [2]:
import pandas as pd 

In [3]:
from aind_data_access_api.document_db import MetadataDbClient

API_GATEWAY_HOST = "api.allenneuraldynamics.org"
DATABASE = 'metadata_index'
COLLECTION = 'data_assets'

docdb_api_client = MetadataDbClient(
   host=API_GATEWAY_HOST,
   database=DATABASE,
   collection=COLLECTION,
)
print(docdb_api_client._base_url)

https://api.allenneuraldynamics.org/v1/metadata_index/data_assets


In [4]:
aggregate = [
  {
    "$match": {
      "session.session_type": "BCI single neuron stim",
      "data_description.data_level": "derived",
      "processing.processing_pipeline.data_processes.start_date_time":{"$gte":"2025-07-21"}
    }
  },
  {
    "$project": {
      "name": 1,
      "subject_id": "$data_description.subject_id",
      "session_time": "$session.session_start_time", 
      "session_type": "$session.session_type", 
      "genotype": "$subject.genotype", 
      "virus_injection_materials": "$procedures.subject_procedures.procedures.injection_materials.name",
      "session_number": {
        "$filter": {
          "input": "$session.stimulus_epochs",
          "as": "epoch",
          "cond": { "$eq": ["$$epoch.stimulus_name", "single neuron BCI conditioning"] }
        }
      },
      'ophys_fov': {
            '$map': {
                'input': '$session.data_streams',
                'as': 'stream',
                'in': {
                    '$map': {
                            'input': '$$stream.ophys_fovs',
                            'as': 'fov',
                            'in': '$$fov.notes'
                    }
                }
            }
        }

    }
  },
  {
    "$project": {
      "name": 1,
      "subject_id": 1,
      "session_time": 1, 
      "session_type": 1,
      "genotype": 1,
      "virus_injection_materials": 1,
      "session_number": { "$arrayElemAt": ["$session_number.session_number", 0] },
      "ophys_fov": 1,
    }
  },
  {'$unwind': {'path': '$ophys_fov', 'preserveNullAndEmptyArrays': False}},
  {'$unwind': {'path': '$ophys_fov', 'preserveNullAndEmptyArrays': False}},
  {'$unwind': {'path': '$virus_injection_materials', 'preserveNullAndEmptyArrays': False}}, 
  {'$unwind': {'path': '$virus_injection_materials', 'preserveNullAndEmptyArrays': False}},
  {'$unwind': {'path': '$virus_injection_materials', 'preserveNullAndEmptyArrays': False}}
  
]

records = docdb_api_client.aggregate_docdb_records(
    pipeline = aggregate,
)


In [5]:
bci_df = pd.DataFrame(records)
bci_df.drop_duplicates(inplace=True)

In [49]:
bci_df.to_csv('/data/metadata.csv', index= False)